# **Importamos la librerias necesarias**

In [1]:
from _utils import *

# **Ahora cargamos los dataset necesarios**

In [17]:
df_coins = pd.DataFrame(read_json("../../data/json/coingecko/new_coins.json"))

# **Inspeccion de Datos y Preprocesamiento de los Datos**


Ahora le damos una vista general de los datos y se limpiaran los datos 

`df_coins`

In [18]:
df_coins.head(5)

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated,category,category_id,exchange_centralized,exchange_id
0,illuvium,ilv,Illuvium,https://assets.coingecko.com/coins/images/1446...,101.140000,648969604.0,132.0,973810125.0,7296580.0,105.470000,...,2021-11-30T15:18:57.382Z,29.230000,246.81593,2021-06-22T14:11:59.151Z,None,2024-04-29T03:05:17.714Z,Adventure Games,adventure-games,True,illuvium
1,aurory,aury,Aurory,https://assets.coingecko.com/coins/images/1932...,0.529247,29028107.0,924.0,52789276.0,243170.0,0.547194,...,2021-11-08T16:04:04.494Z,0.310679,70.45106,2022-11-18T22:18:37.175Z,None,2024-04-29T03:05:05.806Z,Adventure Games,adventure-games,True,aurory
2,bullieverse,bull,Bullieverse,https://assets.coingecko.com/coins/images/2417...,0.007393,5328881.0,1773.0,7396653.0,70911.0,0.007743,...,2022-03-09T13:10:35.955Z,0.000731,909.13361,2023-08-19T02:45:15.229Z,None,2024-04-29T03:05:00.575Z,Adventure Games,adventure-games,True,bullieverse
3,eldarune,elda,Eldarune,https://assets.coingecko.com/coins/images/3033...,0.011672,2447421.0,2188.0,6887250.0,356979.0,0.012363,...,2024-01-02T08:40:27.468Z,0.002532,358.39700,2023-09-26T08:18:38.855Z,None,2024-04-29T03:05:18.145Z,Adventure Games,adventure-games,True,eldarune
4,mch-coin,mchc,MCH Coin,https://assets.coingecko.com/coins/images/1539...,0.068604,2347623.0,2209.0,2418531.0,63599.0,0.070353,...,2021-07-30T05:15:27.404Z,0.022968,198.90621,2022-06-18T20:17:17.271Z,None,2024-04-29T03:04:39.536Z,Adventure Games,adventure-games,True,mch-coin


In [19]:
df_coins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8453 entries, 0 to 8452
Data columns (total 30 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                8453 non-null   object 
 1   symbol                            8453 non-null   object 
 2   name                              8453 non-null   object 
 3   image                             8453 non-null   object 
 4   current_price                     8451 non-null   float64
 5   market_cap                        8452 non-null   float64
 6   market_cap_rank                   4087 non-null   float64
 7   fully_diluted_valuation           7800 non-null   float64
 8   total_volume                      8452 non-null   float64
 9   high_24h                          8050 non-null   float64
 10  low_24h                           8050 non-null   float64
 11  price_change_24h                  8050 non-null   float64
 12  price_

In [20]:
# Cuantas monedas de exchanges centralizados y decentralizados
df_coins[df_coins["exchange_centralized"] == True].shape[0], df_coins[df_coins["exchange_centralized"] == False].shape[0]

(4446, 4007)

## Casos especiales

```json
{
	"id": "mother-of-memes-2",
	"symbol": "haha",
	"name": "Mother of Memes",
	"image": "https://assets.coingecko.com/coins/images/36245/large/HAHA_Logo.png?1710921482",
	"current_price": null,
	"market_cap": null,
	"market_cap_rank": null,
	"fully_diluted_valuation": null,
	"total_volume": 0.0,
	"high_24h": null,
	"low_24h": null,
	"price_change_24h": null,
	"price_change_percentage_24h": null,
	"market_cap_change_24h": null,
	"market_cap_change_percentage_24h": null,
	"circulating_supply": null,
	"total_supply": 100000000000000.0,
	"max_supply": 100000000000000.0,
	"ath": null,
	"ath_change_percentage": 0.0,
	"ath_date": null,
	"atl": null,
	"atl_change_percentage": 0.0,
	"atl_date": null,
	"roi": null,
	"last_updated": "2024-04-29T03:03:26.289Z",
	"category": "Meme",
	"category_id": "meme-token"
}
```

In [21]:
find_columns_with_nulls(df_coins)

[('current_price', 2),
 ('market_cap', 1),
 ('market_cap_rank', 4366),
 ('fully_diluted_valuation', 653),
 ('total_volume', 1),
 ('high_24h', 403),
 ('low_24h', 403),
 ('price_change_24h', 403),
 ('price_change_percentage_24h', 403),
 ('market_cap_change_24h', 405),
 ('market_cap_change_percentage_24h', 405),
 ('circulating_supply', 1),
 ('total_supply', 547),
 ('max_supply', 2443),
 ('ath', 4),
 ('ath_date', 4),
 ('atl', 4),
 ('atl_date', 4),
 ('roi', 8178),
 ('last_updated', 1)]